# This Notebook is a simple Demonstration of how to use better system prompting to improve your output

In [2]:
from dotenv import load_dotenv
load_dotenv()
from anthropic import Anthropic

client = Anthropic()
model = "claude-sonnet-4-0"

# Building Helper Functions

In [3]:
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)

def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature
    }
    
    if system:
        params["system"] = system
    
    message = client.messages.create(**params)
    return message.content[0].text

# Now Lets try to solve a math problem without system prompt

In [ ]:
messages = []
while True:
    user_input = input("> ")
    if user_input == "exit":
        break
    print("> ", user_input)
    add_user_message(messages, user_input)
    answer = chat(messages)
    add_assistant_message(messages, answer)
    print('---')
    print(answer)
    print('---')

>  How Do I solve for x in 5x + 3 = 2 
---
I'll help you solve for x in the equation 5x + 3 = 2.

The goal is to isolate x on one side of the equation. Here are the steps:

**Step 1:** Subtract 3 from both sides
5x + 3 - 3 = 2 - 3
5x = -1

**Step 2:** Divide both sides by 5
5x ÷ 5 = -1 ÷ 5
x = -1/5

**Answer:** x = -1/5 (or x = -0.2)

**Check your work:**
Substitute x = -1/5 back into the original equation:
5(-1/5) + 3 = -1 + 3 = 2 ✓

The solution is correct!
---


# Now Lets try to stream an answer instead of waiting for the model to finish answering

In [12]:
messages = []
while True: 
    user_input = input("> ")
    if user_input == "exit":
        break
    print("> ", user_input)

    add_user_message(messages, user_input)

    print('---')
    with client.messages.stream(
        model=model,
        max_tokens=1000,
        messages=messages
    ) as stream:
        for text in stream.text_stream:
            # Send each chunk to your client
            print(text, end="")
        
        # Get the complete message for database storage
        final_message = stream.get_final_message()

    print('\n---')

    
    answer = final_message.content[0].text
    add_assistant_message(messages, answer)


>  Hello
---
Hello! How are you doing today? Is there anything I can help you with?
---
>  what is the 10th prime number?
---
The 10th prime number is 29.

Here's the sequence of the first 10 prime numbers:
1st: 2
2nd: 3
3rd: 5
4th: 7
5th: 11
6th: 13
7th: 17
8th: 19
9th: 23
10th: 29
---
>  explain your logic
---
I found the 10th prime number by systematically identifying prime numbers in order.

Here's my logic:

**Definition**: A prime number is a natural number greater than 1 that has no positive divisors other than 1 and itself.

**Process**: I went through the natural numbers starting from 2 and checked each one:

- **2**: Only divisible by 1 and 2 → Prime #1
- **3**: Only divisible by 1 and 3 → Prime #2
- **4**: Divisible by 1, 2, and 4 → Not prime
- **5**: Only divisible by 1 and 5 → Prime #3
- **6**: Divisible by 1, 2, 3, and 6 → Not prime
- **7**: Only divisible by 1 and 7 → Prime #4

I continued this pattern, testing each number for divisibility by all numbers less than itself